In [75]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import os
import argparse


# In[2]:


MemSize = 1000 # memory size, in reality, the memory size should be 2^32, but for this lab, for the space resaon, we keep it as this large number, but the memory is still 32-bit addressable.


# In[3]:




In [1330]:

# In[80]:


class register(object): #Used  to represent n-bit binary digits in general but considers the finite word effects
    #Used as a super class to do the data conversion and operations throughout the program
    
    def __init__(self, data=0,   **kwargs): 
        #kwargs is used to accept an input value if specified or place a default value
        
        #register full size if stated explicitly, this is the total number of bits in the register represention
        n = kwargs.get('n')
        
        #Endianess of the input if the input is provided as a byte array/list or a large binary number
        self.endian = kwargs.get('endian','big')
        
        #debug is used to activate the printing
        self.debug = kwargs.get('debug',False)
        
        #ByteAddressable, used to display bytes in 8-bit/2byte memory width 
        # or more/less only , used to calculate the big/little endian form of the register
        self.memoryAddressable = kwargs.get('memoryAddressable',8) 
        
        #If the input provided is signed or not, this will impact the integer equivalent, if signed then 2-comp is used
        self.signed = kwargs.get('signed',False)
        
        #If the input is provided as a Hex digits list like ['F1','3C',...] instead of a binary digits list ['1001',..]
        self.hex = kwargs.get('Hex',False)
        
        # width, used only if data is read in arrays of binary digits to specify the length of each array element
        # i.e. register(['1','11']) by default will use 8 bits for each number, if we need other than 8-bits per number
        # then we use width to zero-Extend the input to the required width
        # e.g. register(['100','11'], width =4) = '0100 0011'
        self.width = kwargs.get('width',8)
        
        #self.isSigned()
        self.sizeOf(data, n=n)
        self.overflow()
        
        if type(data)==int: #Loads an integer number
            #Does not depend on Endianess since the data is input as integer
            
            #How python stores the data, or aka the bits form translated into integer depending on signed or not
            self.data = data 
            
            
        elif type(data)==bytearray: #loads a byte array object, e.g. bytearray([1,2])
            #will depend on endianess since the input is a byte array objectr
            #Will depend on sign to convert to binary
            self.data=int.from_bytes(data, byteorder=self.endian, signed=self.signed)

        elif type(data)==str and not self.hex:  #loads a string of binary digits, e.g. '100011'
            
            data=data.replace(" ","").replace("	","").replace("_","").replace("0b","").replace("-","")
            self.data=int(data, 2)
            
        elif type(data)==str and self.hex: #loads a string of hex digits, e.g. '1F0F'

            data=data.replace(" ","").replace("	","").replace("_","").replace("0x","").replace("-","")
            self.data=int(data, 16)

        elif type(data)==list and not self.hex: #loads a list of binary digits, e.g. ['10', '1001']
            # Depends on endianess
            # Requires a width parameter to determine the width for every digit, supports width >=8 in this verision
            
            #convert the binary list to int list
            dataIntList = [int(i.replace(" ","").replace("	","").replace("_","").replace("0b","").replace("-",""),2) for i in data]
            
            #convert the int list to bytes array, min width is 1 byte per digit
            #since bytearray is limited to 0-255 integers so we use the list conversion first to construct the byte array
            dataBytesList = [dataInt.to_bytes(length=(self.width+1//8) if self.width//8==0 else self.width//8, byteorder=self.endian) for dataInt in dataIntList]
            dataBytesList=b''.join(dataBytesList)
            
            #convert the bytearray to an integer
            self.data=int.from_bytes(dataBytesList, byteorder=self.endian, signed=self.signed)

        elif type(data)==list and self.hex: #loads a string of hex digits e.g. ['1F','3FF']
            #Handled similar to the string of bits but with a different base
            
            dataIntList = [int(i.replace(" ","").replace("	","").replace("_","").replace("0x",'').replace("-",''),16) for i in data]
            dataBytesList = [dataInt.to_bytes(length=(self.width+1//8) if self.width//8==0 else self.width//8, byteorder=self.endian) for dataInt in dataIntList]
            dataBytesList=b''.join(dataBytesList)

            self.data=int.from_bytes(dataBytesList, byteorder=self.endian, signed=self.signed)

                                
        self.sizeMask() #Generate binary masks with size n-bits to display the numbers correctly in python
        self.intToBinary() #Generate the binary equivalent of the whole n-bit register, dependent on the sign
        self.signSymb() #Determine the sign of the number , -1 for -ve and 1 for +ve
        self.binToMemoryBytes()  #Generate botht little/big endian forms array for the register, if needed in other operations
        self.intToHex() #Generate the Hex string representation
        self.binToInt() #Generate the signed and unsigned integer for the register, self.data is the unsigned and self.dataInt is the signed
    
    def sizeOf(self , data, **kwargs): #Determine the minimal n-bits nmin required for representing the register
        #if n-bits register size is explicitly provided, then detect the overflow and use the n-bits insead of n-min
        
        if type(data)==int : 
            self.nmin = len(bin(data).replace("0b","")) if self.signed==False else len(bin(abs(data)).replace("0b",""))+1 

        elif type(data)==bytearray: 
            self.nmin = int(len(data)*8)

        elif type(data)==str and not self.hex:  
            self.nmin = int(len(data))

        elif type(data)==str and self.hex:  
            self.nmin = int(len(data)*4)

        elif type(data)==list and not self.hex: 
            self.nmin = int(len(data)*self.width)
        elif type(data)==list and self.hex: 
            self.nmin = int(len(data)*self.width)
            
        if kwargs.get('n'): #Use the provided n-bits OR just use the min required n-bits for the register
            self.n = kwargs.get('n')
        else:
            self.n = self.nmin 
            
        #No. of bytes required to represent the register, this is going to be used in the memory representation
        self.nBytes = (self.n//8) if self.n%8 ==0 else (self.n//8)+1

    def overflow(self, **kwargs):
            #If n is provided but it is shorter than the required n-bits, the raise an overflow flag
        if self.n  < self.nmin:
            self.v=1
        else:
            self.v=0
        
    def sizeMask(self): #Generate a mask of n-bit '1's to be used in displaying the bits correctly
        self.mask= (2**(self.n))-1 #Mask is all 1s in integer
        self.maskBin = bin(self.mask).replace("0b","") #Mask in bits
        self.maskMemory= (2**(self.nBytes * 8))-1 #All 1s in integer
        self.maskBinMemory = bin(self.maskMemory).replace("0b","") #All 1s in bits
      
    def intToBinary(self): #Generate the n-bit binary form of the register, with and without the python prefix 0b
        
        self.dataBin = f"{self.data & self.mask :0{self.n}b}"#.replace("-","1") #data in bits, to be used for slicing
        self.dataBinPre = f"{bin(self.data & self.mask)}"

        
    def intToHex(self): #Generate the n-bit hex form of the register, with and without the python prefix 0x
        self.dataHex = f"{self.data & self.mask :0{(self.n//4 if self.n%4==0 else (self.n//4)+1 )}X}" 
        self.dataHexPre = f"{hex(self.data & self.mask)}"

        #data in Hex, ignores the sign, and adds a hex bit to represent values that are not divisible by 4
        #, i.e. for n=5 bits use 2 hex digits to view the Hex form as 2x4=8 bits = 2 Hex digits
        self.dataHex = f"{self.data & self.mask :0{(self.n//4 if self.n%4==0 else (self.n//4)+1 )}X}" 
        self.dataHexPre = f"{hex(self.data & self.mask)}"

        
    def binToInt(self): #Generate the signed binary representation, depends on the sign bit deduced from the self.signed attrib
        self.dataInt= (int(self.dataBin[0]) * (self.sign * 2**(self.n-1)) + int(self.dataBin[1:],2))  if self.n>=2 else int(self.dataBin,2)

    def binToMemoryBytes(self):
        #self.memoryAddressable is used here mainly to change the form of the words to be compatible with
        # 8-bit/32-bit/n-bit addressable memories
        
        # Display the n-bits proerply by extending them using the n-bit mask to get an 8n-bits (n-bytes) string.
        self.dataBinMemoryBytesExt = f"{self.data & self.maskMemory :0{self.nBytes * 8}b}" 
        
        #couple the bit array each n-bytes togther, and change their order to generate both the big/little forms in the output
        self.dataBinMemoryBytesLittle = [self.dataBinMemoryBytesExt[i:i+self.memoryAddressable] for i in range(0,self.nBytes*8,self.memoryAddressable)][::-1]
        self.dataBinMemoryBytesBig = [self.dataBinMemoryBytesExt[i:i+self.memoryAddressable] for i in range(0,self.nBytes*8,self.memoryAddressable)]
    
        #Repeat for Hex
        self.dataHexMemoryBytesExt = f"{self.data & self.maskMemory :0{self.nBytes * 2}X}" 
        self.dataHexMemoryBytesLittle = [self.dataHexMemoryBytesExt[i:i+(self.memoryAddressable//4)] for i in range(0,self.nBytes*2,(self.memoryAddressable//4))][::-1]
        self.dataHexMemoryBytesBig = [self.dataHexMemoryBytesExt[i:i+(self.memoryAddressable//4)] for i in range(0,self.nBytes*2,(self.memoryAddressable//4))]

    def signSymb(self): #Determine the sign bit, 1 for +ve and -1 for -ve
        self.sign= -1 if (self.signed and int(self.dataBin[0])==1) else 1
                    
    def __len__(self): #Return the size of the register
        return self.n
    
    def __getitem__(self,index): #return the i-th bit of the register, compatible with bit order 0->(n-1),
        #Used to allow python style lists slicing with the bit order properly
        return self.dataBin[abs(index-(self.n-1))]
    
    def bit(self,start,*args,**kwargs): #Returns the bit(start, end) or bit(start), used for decoding instructions
        if args: #if input is provided as register.bit(4) then return the 4-th bit only, so return register[3]
            end=args[0]
        else:
            end=start
        sliceBin = [self.__getitem__(i) for i in range(start,end+1,1) ]
        
        return "".join(sliceBin[::-1])

    def __str__(self): #Use the print statement to show all attributes of the register
        return f"{vars(self)}"
    
    def signExt(self, n): #sign Extend the register to a new size n-bit, by re-initializing the digit as signed
        self.n=n
        self.__init__(data=self.data, n=self.n , endian=self.endian 
                      , debug=self.debug, memoryAddressable=self.memoryAddressable
                     , signed= self.signed, Hex = self.hex
                     )
    def zeroExt(self, n): #zero fill the register
        self.n=n
        self.__init__(data=self.data, n=self.n , endian=self.endian 
                      , debug=self.debug, memoryAddressable=self.memoryAddressable
                     , signed= False, Hex = self.hex
                     )
    def signBin(self, signed=False): #convert from unsigned to signed
        self.__init__(data=self.data, n=self.n , endian=self.endian 
              , debug=self.debug, memoryAddressable=self.memoryAddressable
             , signed= signed, Hex = self.hex
             )


        #sign-aware operations, with output size as the size of the largest register.
        #it is assumed that the destination register size will be the same as the largest operand register.
    def __add__(self, self2): #signed addition, both operands are sign extended, and the size of the destination register
        # is the size of the smallest register
        n_new=max(self.n , self2.n)
        
        self.signExt(n_new)
        self2.signExt(n_new)
        
        dataInt = self.dataInt + self2.dataInt
        return register(data=dataInt, n=n_new , endian=self.endian 
                      , debug=self.debug, memoryAddressable=self.memoryAddressable
                        , signed= self.signed, Hex = self.hex
                         )
    

        
    def __sub__(self, self2):
        self.n=max(self.n , self2.n)

        self.signExt(self.n)
        self2.signExt(self.n)

        self.dataInt = self.dataInt - self2.dataInt
        return register(data=self.dataInt, n=self.n , endian=self.endian 
                      , debug=self.debug, memoryAddressable=self.memoryAddressable
                         , signed= self.signed, Hex = self.hex
                         )

    def __or__(self, self2):
        self.n=max(self.n , self2.n)

        self.signExt(self.n)
        self2.signExt(self.n)

        self.dataInt = self.dataInt | self2.dataInt
        return register(data=self.dataInt, n=self.n , endian=self.endian 
                      , debug=self.debug, memoryAddressable=self.memoryAddressable
                         , signed= self.signed, Hex = self.hex
                         )

    def __and__(self, self2):
        self.n=max(self.n , self2.n)

        self.signExt(self.n)
        self2.signExt(self.n)

        self.dataInt = self.dataInt & self2.dataInt
        return register(data=self.dataInt, n=self.n , endian=self.endian 
                      , debug=self.debug, memoryAddressable=self.memoryAddressable
                         , signed= self.signed, Hex = self.hex
                         )
    
    def __xor__(self, self2):
        self.n=max(self.n , self2.n)

        self.signExt(self.n)
        self2.signExt(self.n)

        self.dataInt = self.dataInt ^ self2.dataInt
        return register(data=self.dataInt, n=self.n , endian=self.endian 
                      , debug=self.debug, memoryAddressable=self.memoryAddressable
                         , signed= self.signed, Hex = self.hex
                         )
    
    def __ne__(self, self2):
        self.n=max(self.n , self2.n)

        self.signExt(self.n)
        self2.signExt(self.n)

        if self.dataInt != self2.dataInt:
            return True
        else:
            return False
        
    def __eq__(self, self2):
        self.n=max(self.n , self2.n)

        self.signExt(self.n)
        self2.signExt(self.n)

        if self.dataInt == self2.dataInt:
            return True
        else:
            return False


In [1331]:



# In[15]:


class instructionRegister(register ): #Class used to decode and execute the instructions
    #We provide PC+4 to the instruction, that is why there is a -4 for jump instructions
    #There instructionRegister will generate Halted and PC attributes that will be fed to the next state object
    def __init__(self, data, PC, halted , r, d):
        super().__init__(data=data, n=32, endian='big', debug=False, memoryAddressable=8, signed=True, hex=False)
        
        #rdInt is the integer of Destination Register
        #r1R is the register class of the contents of the source register1
        #r1R.dataBin is the bit representation
        
        
        self.PC = PC
        self.halted = halted
        self.opcode=self.bit(0,6)
        r.writeMemoryPtrBuffer(data=['0'*32], writeAddress=0,  offset=0, width=32) #Clear x0
        
        if self.opcode == '0110011':
            self.format='R'
            self.RDecode(r,d)
        elif self.opcode == '0010011' or self.opcode=='0000011':
            self.format='I'
            self.IDecode(r,d)
        elif self.opcode == '1101111':
            self.format='J'
            self.JDecode(r,d)
        elif self.opcode == '1100011':
            self.format='B'
            self.BDecode(r,d)
        elif self.opcode == '0000011':
            self.format='I'
            self.IDecode(r,d)
        elif self.opcode == '0100011':
            self.format='S'
            self.SDecode(r,d)
        elif self.opcode == '1111111':
            self.format='-'
            self.op='HALT'
            self.halted = True
            self.PC = self.PC - 4
            print (f"Instructions Halted, Halted = {self.halted}, emulated by x0+x0=x0")
            self.instrStr=f"format = {self.format}, opname={self.op}, opcode={self.opcode}"
            self.rdInt = 0
            self.rs1Int = 0
            self.rs1R = r.readMemoryPtrBuffer(self.rs1Int) if self.rs1Int !=0 else registerRISC(0)
            self.rs2Int = 0
            self.rs2R = r.readMemoryPtrBuffer(self.rs2Int) if self.rs2Int !=0 else registerRISC(0)
            self.R_ADD(  r, d)

        print (f"Instruction String = {self.instrStr}")
        self.clearx0(r)
        
    def RDecode(self, r, d):
        self.func3=self.bit(12,14)
        
        self.rd=self.bit(7,11)
        self.rdInt=int(self.rd,2)
        
        self.rs1=self.bit(15,19)
        self.rs1Int=int(self.rs1,2)
        self.rs1R = r.readMemoryPtrBuffer(self.rs1Int) if self.rs1Int !=0 else registerRISC(0)
        
        self.rs2=self.bit(20,24)
        self.rs2Int=int(self.rs2,2)
        self.rs2R = r.readMemoryPtrBuffer(self.rs2Int) if self.rs2Int !=0 else registerRISC(0)
        
        self.func7=self.bit(25,31)
        
        if self.func3 =='000' and self.func7=='0000000':
            self.op = 'ADD'
            self.execute = self.R_ADD(r,d)
        elif self.func3=='000' and self.func7=='0100000':
            self.op = 'SUB'
            self.execute = self.R_SUB(r,d) 
        elif self.func3=='100' and self.func7=='0000000':
            self.op = 'XOR'
            self.execute = self.R_XOR(r,d)
        elif self.func3=='110' and self.func7=='0000000':
            self.op = 'OR'
            self.execute = self.R_OR(r,d) 
        elif self.func3=='111' and self.func7=='0000000':
            self.op = 'AND'
            self.execute = self.R_AND(r,d) #NOT TESTED
         
        self.instrStr= f"format = {self.format}, opname={self.op}, opcode={self.opcode},rs1={self.rs1}, rs2={self.rs2}, rd={self.rd}, func3={self.func3}, func7={self.func7}"
        self.execute
        
        
    def IDecode(self ,  r, d):
        self.rd=self.bit(7,11)
        self.rdInt=int(self.rd,2)

        self.func3=self.bit(12,14)
        
        self.rs1=self.bit(15,19)
        self.rs1Int=int(self.rs1,2)
        self.rs1R = r.readMemoryPtrBuffer(self.rs1Int) if self.rs1Int !=0 else registerRISC(0)

        
        self.imm=self.bit(20,31)
        #self.immInt=int(self.imm,2)
        #self.immR=registerRISC(data=self.imm)
        self.immR=registerRISC(register(self.imm,signed=True).dataInt)
        self.immInt=self.immR.dataInt
        
        if self.func3=='000' and self.opcode=='0010011':
            self.op = 'ADDI'
            self.execute = self.I_ADDI(r,d)
        elif self.func3=='100' :
            self.op = 'XORI'
            self.execute = self.I_XORI(r,d)
        elif self.func3=='110' :
            self.op = 'ORI'
            self.execute = self.I_ORI(r,d)
        elif self.func3=='111':
            self.op = 'ANDI'
            self.execute = self.I_ANDI(r,d)
        elif self.func3=='000' and self.opcode=='0000011':
            self.op = 'LW'
            self.execute = self.I_LW(r,d)
            
        self.instrStr= f"format = {self.format}, opname={self.op}, opcode={self.opcode},rs1={self.rs1},  rd={self.rd}, func3={self.func3}, imm={self.imm}"
        self.execute
        #self.clearx0(r)

    def SDecode(self ,  r, d):
        self.imm=self.bit(25,31) + self.bit(7,11)
        #self.immInt=int(self.imm,2)
        #self.immR=registerRISC(data=self.imm)
        self.immR=registerRISC(register(self.imm,signed=True).dataInt)
        self.immInt=self.immR.dataInt
        
        self.func3=self.bit(12,14)
        
        self.rs1=self.bit(15,19)
        self.rs1Int=int(self.rs1,2)
        self.rs1R = r.readMemoryPtrBuffer(self.rs1Int) if self.rs1Int !=0 else registerRISC(0)

        self.rs2=self.bit(20,24)
        self.rs2Int=int(self.rs2,2)
        self.rs2R = r.readMemoryPtrBuffer(self.rs2Int) if self.rs2Int !=0 else registerRISC(0)
        
        if self.func3=='010':
            self.op = 'SW'
            self.execute = self.S_SW(r,d)

        self.instrStr= f"format = {self.format}, opname={self.op}, opcode={self.opcode},rs1={self.rs1}, rs2={self.rs2}, func3={self.func3},  imm={self.imm}"
        self.execute
        
    def BDecode(self ,  r, d):
        #self.imm= self.bit(7)+self.bit(7)+self.bit(25,30)+self.bit(8,11)+'0'
        self.imm= self.bit(31)+self.bit(7)+self.bit(25,30)+self.bit(8,11)+'0'
        #self.immInt=int(self.imm,2)
        #elf.immR=registerRISC(data=self.imm)
        self.immR=registerRISC(register(self.imm,signed=True).dataInt)
        self.immInt=self.immR.dataInt

        self.func3=self.bit(12,14)

        self.rs1=self.bit(15,19)
        self.rs1Int=int(self.rs1,2)
        self.rs1R = r.readMemoryPtrBuffer(self.rs1Int) if self.rs1Int !=0 else registerRISC(0)

        self.rs2=self.bit(20,24)
        self.rs2Int=int(self.rs2,2)
        self.rs2R = r.readMemoryPtrBuffer(self.rs2Int) if self.rs2Int !=0 else registerRISC(0)

        if self.func3=='000':
            self.op = 'BEQ'
            self.execute = self.B_BEQ(r,d) #NOT TESTED
        elif self.func3=='001' :
            self.op = 'BNE'
            self.execute = self.B_BNE(r,d)
        self.instrStr= f"format = {self.format}, opname={self.op}, opcode={self.opcode},rs1={self.rs1}, rs2={self.rs2}, func3={self.func3},  imm={self.imm}"

        self.execute
        
    def UDecode(self ,  r, d):
        self.rd=self.bit(7,11)
        self.rdInt=int(self.rd,2)

        self.imm=self.bit(12,31)+'0'*12 #12 + 31-12 + 1
        #self.immInt=int(self.imm,2)
        #elf.immR=registerRISC(data=self.imm)
        self.immR=registerRISC(register(self.imm,signed=True).dataInt)
        self.immInt=self.immR.dataInt
        
        self.instrStr= f"format = {self.format}, opname={self.op}, opcode={self.opcode}, rd={self.rd},  imm={self.imm}"
        
        #self.clearx0(r)
        
    def JDecode(self ,  r, d):
        self.rd=self.bit(7,11)
        self.rdInt=int(self.rd,2)
        
        #self.imm= self.bit(31)+self.bit(12,12+8)+self.bit(12+8+1)+self.bit(12+8+1 + 1, 12+8+1 + 1 + 10)+'0'
        #self.imm= self.bit(31)+self.bit(10,18)+self.bit(19)+self.bit(20,30)+'0' #1+10+1+1+8+1+1
        self.imm= self.bit(31)+self.bit(12,19)+self.bit(20)+self.bit(21,30)+'0' #1+10+1+1+8+1+1
        #self.immInt=int(self.imm,2)
        #self.immR=registerRISC(data=self.imm)
        self.immR=registerRISC(register(self.imm,signed=True).dataInt)
        self.immInt=self.immR.dataInt
        
        self.op='JAL'
        self.instrStr= f"format = {self.format}, opname={self.op}, opcode={self.opcode}, rd={self.rd},  imm={self.imm}"
        self.execute = self.J_JAL(r,d)
        self.execute
        #self.clearx0(r)
        
    def printInstr(self):
        #print(f"format = {self.format}, opcode={self.opcode},rs1={self.rs1}, rs2={self.rs2}, rd={self.rd}, func3={self.func3}, func7={self.func7}, imm={self.imm}")

        if self.format == 'R':
            print(f"format = {self.format}, opname={self.op}, opcode={self.opcode},rs1={self.rs1}, rs2={self.rs2}, rd={self.rd}, func3={self.func3}, func7={self.func7}")
        elif self.format == 'I':
            print(f"format = {self.format}, opname={self.op}, opcode={self.opcode},rs1={self.rs1},  rd={self.rd}, func3={self.func3}, imm={self.imm}")
        elif self.format == 'S':
            print(f"format = {self.format}, opname={self.op}, opcode={self.opcode},rs1={self.rs1}, rs2={self.rs2}, func3={self.func3},  imm={self.imm}")
        elif self.format == 'B':
            print(f"format = {self.format}, opname={self.op}, opcode={self.opcode},rs1={self.rs1}, rs2={self.rs2}, func3={self.func3},  imm={self.imm}")
        elif self.format == 'U':
            print(f"format = {self.format}, opname={self.op}, opcode={self.opcode}, rd={self.rd},  imm={self.imm}")
        elif self.format == 'J':
            print(f"format = {self.format}, opname={self.op}, opcode={self.opcode}, rd={self.rd},  imm={self.imm}")
        elif self.format == '-':
            print(f"format = {self.format}, opname={self.op}, opcode={self.opcode}")

    def S_SW(self ,  r, d):
        #print(f"rs2 {self.rs2R}")
        print(f"\t op={self.op} rs2={self.rs2Int}, imm={self.immInt}(rs1={self.rs1Int})")
        #self.temp = d.readMemoryPtrBuffer(self.rs2R.data)
        self.temp = r.readMemoryPtrBuffer(self.rs2Int )#, width=32)
        print(f"\t M[rs2]=  {self.temp.dataHexMemoryBytesBig} = {self.temp.dataBin} = {self.temp.dataInt} ")
        print(f"\t temp=  {[self.temp.dataBinMemoryBytesBig]} ")
        d.writeMemoryPtrBuffer(data=self.temp.dataBinMemoryBytesBig, writeAddress=self.rs1Int,  offset=self.immInt)

    def I_LW(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, imm={self.immInt}(r1={self.rs1Int})")
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        self.temp = d.readMemoryPtrBuffer(self.rs1Int, offset=self.immInt)
        print(f"\t M[rs1 + imm]=  {self.temp.dataHexMemoryBytesBig} = {self.temp.dataBin} = {self.temp.dataInt} ")
        print(f"\t temp=  {[self.temp.dataBin]} ")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)


        
    def R_ADD(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, r1={self.rs1Int} , r2={self.rs2Int}")
        self.temp = self.rs1R + self.rs2R
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        print(f"\t r1={self.rs1R.dataHexMemoryBytesBig} +\n\t r2={self.rs2R.dataHexMemoryBytesBig} =\n\t rd={self.temp.dataHexMemoryBytesBig} ")
        #print(f"\t temp=  {[self.temp.dataBin]} ")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)

    def R_SUB(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, r1={self.rs1Int} , r2={self.rs2Int}")
        self.temp = self.rs1R - self.rs2R
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        print(f"\t r1={self.rs1R.dataHexMemoryBytesBig} +\n\t r2={self.rs2R.dataHexMemoryBytesBig} =\n\t rd={self.temp.dataHexMemoryBytesBig} ")
        #print(f"\t temp=  {[self.temp.dataBin]} ")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)

        #NOT Tested
    def R_AND(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, r1={self.rs1Int} , r2={self.rs2Int}")
        self.temp = self.rs1R & self.rs2R
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        print(f"\t r1={self.rs1R.dataHexMemoryBytesBig} +\n\t r2={self.rs2R.dataHexMemoryBytesBig} =\n\t rd={self.temp.dataHexMemoryBytesBig} ")
        #print(f"\t temp=  {[self.temp.dataBin]} ")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)

    def R_OR(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, r1={self.rs1Int} , r2={self.rs2Int}")
        self.temp = self.rs1R | self.rs2R
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        print(f"\t r1={self.rs1R.dataHexMemoryBytesBig} +\n\t r2={self.rs2R.dataHexMemoryBytesBig} =\n\t rd={self.temp.dataHexMemoryBytesBig} ")
        #print(f"\t temp=  {[self.temp.dataBin]} ")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)

    def R_XOR(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, r1={self.rs1Int} , r2={self.rs2Int}")
        self.temp = self.rs1R ^ self.rs2R
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        print(f"\t r1={self.rs1R.dataHexMemoryBytesBig} +\n\t r2={self.rs2R.dataHexMemoryBytesBig} =\n\t rd={self.temp.dataHexMemoryBytesBig} ")
        #print(f"\t temp=  {[self.temp.dataBin]} ")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)

        
    def clearx0(self, r):
        print("\n Clearing x0")
        r.writeMemoryPtrBuffer(data=['0'*32], writeAddress=0,  offset=0, width=32)

    def I_ADDI(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, imm={self.immInt}(r1={self.rs1Int})")
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        #self.temp = d.readMemoryPtrBuffer(self.rs1Int, offset=0) + registerRISC(self.immInt)
        #self.temp = d.readMemoryPtrBuffer(self.rs1Int, offset=0) + self.immR.dataInt
        self.temp = self.rs1R  + self.immR
        print(f"\t imm=\t{self.immR.dataBin} = {self.immR.dataInt}")
        print(f"\t +R1=\t{self.rs1R.dataBin} = {self.rs1R.dataInt}")
        print(f"\t temp=\t{self.temp.dataBin} = {self.temp.dataInt}")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)
        

    def I_ORI(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, imm={self.immInt}(r1={self.rs1Int})")
        #print(f"\t op={self.op} rd={self.rdInt}, imm={self.immInt}(r1={self.rs1Int})")
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        #self.temp = d.readMemoryPtrBuffer(self.rs1Int, offset=0) + registerRISC(self.immInt)
        #self.temp = d.readMemoryPtrBuffer(self.rs1Int, offset=0) + self.immR.dataInt
        self.temp = self.rs1R  | self.immR
        print(f"\t imm=\t{self.immR.dataBin} = {self.immR.dataInt}")
        print(f"\t +R1=\t{self.rs1R.dataBin} = {self.rs1R.dataInt}")
        print(f"\t temp=\t{self.temp.dataBin} = {self.temp.dataInt}")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)
        
    def I_ANDI(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, imm={self.immInt}(r1={self.rs1Int})")
        #print(f"\t op={self.op} rd={self.rdInt}, imm={self.immInt}(r1={self.rs1Int})")
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        #self.temp = d.readMemoryPtrBuffer(self.rs1Int, offset=0) + registerRISC(self.immInt)
        #self.temp = d.readMemoryPtrBuffer(self.rs1Int, offset=0) + self.immR.dataInt
        self.temp = self.rs1R  & self.immR
        print(f"\t imm=\t{self.immR.dataBin} = {self.immR.dataInt}")
        print(f"\t +R1=\t{self.rs1R.dataBin} = {self.rs1R.dataInt}")
        print(f"\t temp=\t{self.temp.dataBin} = {self.temp.dataInt}")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)
        
    def I_XORI(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, imm={self.immInt}(r1={self.rs1Int})")
        #print(f"\t op={self.op} rd={self.rdInt}, imm={self.immInt}(r1={self.rs1Int})")
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        #self.temp = d.readMemoryPtrBuffer(self.rs1Int, offset=0) + registerRISC(self.immInt)
        #self.temp = d.readMemoryPtrBuffer(self.rs1Int, offset=0) + self.immR.dataInt
        self.temp = self.rs1R  ^ self.immR
        print(f"\t imm=\t{self.immR.dataBin} = {self.immR.dataInt}")
        print(f"\t +R1=\t{self.rs1R.dataBin} = {self.rs1R.dataInt}")
        print(f"\t temp=\t{self.temp.dataBin} = {self.temp.dataInt}")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)
        
        #print(f"\t rs1 + imm =  {self.temp.dataHexMemoryBytesBig} = {self.temp.dataBin} = {self.temp.dataInt} ")
        #print(f"\t temp=  {[self.temp.dataBin]} ")
        #r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)

    def B_BNE(self, r,d):
        print(f"\t op={self.op} rs1={self.rs1R.dataBin} rs2 ={self.rs2R.dataBin}, imm={self.immR.dataInt} ")
        print(f"\t Is rs1={self.rs1R.dataBin} NEq to rs2 ={self.rs2R.dataBin} ? {self.rs1R != self.rs2R}")
        if self.rs1R != self.rs2R :
            print(f"\t Not Equal : Branch Taken, PC will change from {self.PC} by offset {self.immR.dataInt} to be = {self.PC + self.immR.dataInt}")
            self.PC = self.PC - 4 + self.immR.dataInt #self.PC is the next state PC+4 
        else:
            print(f"\t Equal : Branch Not Taken, PC will remain from {self.PC}, ignoring the offset {self.immR.dataInt}")

    def B_BEQ(self, r,d): #NOT TESTED
        print(f"\t op={self.op} rs1={self.rs1R.dataBin} rs2 ={self.rs2R.dataBin}, imm={self.immR.dataInt} ")
        print(f"\t Is rs1={self.rs1R.dataBin} Eq to rs2 ={self.rs2R.dataBin} ? {self.rs1R == self.rs2R}")
        if self.rs1R == self.rs2R :
            print(f"\t Equal : Branch Taken, PC will change from {self.PC} by offset {self.immR.dataInt} to be = {self.PC + self.immR.dataInt}")
            self.PC = self.PC - 4 + self.immR.dataInt
        else:
            print(f"\t Not Equal : Branch Not Taken, PC will remain from {self.PC}, ignoring the offset {self.immR.dataInt}")

    def J_JAL(self, r,d): #Assumes input is PC+4
        #self.rdInt=int(self.rd,2)
        #self.immInt=self.immR.dataInt
        #self.temp.dataBin = self.immR.dataInt
        print(f"\t op={self.op} rd={self.rdInt}, imm={self.immInt}")
        print(f"\t Current Next State PC ={self.PC} will be saved in rd={self.rdInt}")
        print(f"\t Then the Current State PC = {self.PC - 4} will jump by offset {self.immR.dataInt} to {self.PC - 4 + self.immR.dataInt}")
        temp = registerRISC(self.PC)
        r.writeMemoryPtrBuffer(data=temp.dataBin ,writeAddress=self.rdInt,  offset=0, width=32)
        self.PC = self.PC - 4 + self.immR.dataInt



In [1332]:

# In[13]:


class Memory( register): 
    #I modified Memory class to inhhert from my register class 
    #it is a class to reprsent all memories AND the register file as well through a unified interface
    
    def __init__(self, name,  memDataFile=True,**kwargs):
        
        self.idx = name #idx is same as id, just renamed to separate it from an id attribute that I used previously
        self.ioDir = ioDir
        self.memDataFile = kwargs.get('memDataFile',False) #whether or not there is a memory data file provided as an input
        self.width = kwargs.get('width', 8)
        self.ioDir = kwargs.get('ioDir', '.')  #Default i/o directory is the current directory
        self.nCellsMax = kwargs.get('nCellsMax', 1000) #Max memory size, used to extend the memory output file with zero padding if required
        self.cycle = kwargs.get('cycle', 0)
        data = kwargs.get('data',[]) #the contents of the memory or registers as a list
        
        self.extend = kwargs.get('extend',False) #Extend the memory list with zero-padding to the max memory size
        
        
        self.inputFileName = kwargs.get('inputFileName',False)
        self.inputFile = kwargs.get('inputFileName',False)
        if self.inputFileName: 
            self.inputFile = os.path.join(self.ioDir , self.inputFileName)
        else:
            self.inputFileName=False
            self.inputFile=False
        self.outputFile = kwargs.get('outputFile',"file.txt")
        
        self.Mem = []
        
        if self.inputFile:
            self.readFile()
        elif data:
            self.Mem = data
            
        if self.extend : self.Mem.extend(['0'*self.width for i in range(self.nCellsMax - len(self.Mem))])
        
        self.__len__()

    def readFile(self):
        with open(self.inputFile) as mem:
            self.Mem = [data.replace("\n", "").replace("\t","") for data in mem.readlines()] ###Test Remove

    def writeFileCycle(self,  **kwargs):
        self.cycle = kwargs.get('cycle', self.cycle)
        self.outputFile = kwargs.get('outputFile' , self.outputFile)
        #resPath = self.ioDir + "\\"  + self.outputFile
        op = ["-"*70+"\n", "State of RF after executing cycle:" + str(self.cycle) + "\n"]
        op.extend([str(val)+"\n" for val in self.Mem])
        if(self.cycle == 0): perm = "w"
        else: perm = "a"
        print (f"printing {self.idx} Memory Cycle writeFileCycle to {self.outputFile}")
        with open(ioDir +  "\\"  + self.outputFile, perm) as file:
            file.writelines(op)
            
    def outputDataMem(self, **kwargs):
        self.outputFile = kwargs.get( 'outputFile' , self.outputFile)
        #resPath = self.ioDir + "\\"  + self.outputFile
        print(f"Printing Data Memory")
        print (f"printing {self.idx} Memory Dump outputDataMem to {self.outputFile}")
        with open(self.ioDir + "\\"  + self.outputFile, "a") as rp:
            #rp.writelines([str(data) + "\n" for data in self.DMem]) xxx I used my object Mem instead of DMEM
            rp.writelines([str(data) + "\n" for data in self.Mem])

            

    def readFromMemory(self, endian='big'): #REads from memory, start/dest address are caluclated by function readMemoryPtrBuffer
        instrBinArr = self.Mem [ self.startAddress : self.endAddress ] if endian=='big' else self.Mem [ self.startAddress : self.endAddress ][::-1]
        print(f"Read Endian={endian} {self.idx} | {self.startAddress}->{self.endAddress-1} | {instrBinArr} ")
        return instrBinArr

    ###Test I added these functions
    def __len__(self):       
        ###Test I added attributes
        self.nByte = len(self.Mem)
        self.nWord = len(self.Mem)//4 
        self.addressMax = len(self.Mem) - 1
        self.addressMaxAlignedOffset = self.addressMax// (32//8) #For 32 bit objects(access) on 8-bit width byte addressable memory
        return self.nByte

    def __str__(self):
        return f"{vars(self)}"
    
    def printMem(self, view='Byte', unit='Bit'):
        print(f"=============\nMemory {self.idx}")
        if view=='Byte' and unit =='Bit':
            for i,byte in enumerate(self.Mem):
                print (f"{i}|{byte}")
        
        elif view=='Byte' and unit =='Hex':
            for i,byte in enumerate(self.Mem):
                print (f"{i}|{format(int(byte,2) , '02X')}")
                
        elif view=='Word'and unit =='Bit':
            for i in range(0,self.nByte,4):
                word = "".join(self.Mem[i : i +4])
                print (f"{i//4}|{word}")

        elif view=='Word' and unit =='Hex':
            for i in range(0,self.nByte,4):
                word = "".join(self.Mem[i : i +4])
                print (f"{i//4}|{format(int(word,2) , '08X')}")
        print(f"=============")

       
        #Calulcates the write start/end addresses then passes control to function writeToMemory
    def writeMemoryPtrBuffer(self, writeAddress, data
                             ,endian='big'
                            ,align=True
                            ,cellOrder=0
                           ,mode='indirect'
                             ,n=32
                             ,offset=0
                             , width = 8 ### xxxx I added this to solve writing to 32 bits large values
                           ): ###Test Calculates Start and end address
        #read 32-bit words aligned by 32-bit/8-width  = 4 by default for 32-bit processors
        #To read unaligned data, use align=False, so the read address will be the then specify the amount of bits as n, 
        #self.orderMemory = ReadAddress
                                    

        if mode == 'indirect':
            self.startAddress = (writeAddress + offset) // (n//self.width)  * (n//self.width)     
            self.endAddress = self.startAddress + (n//self.width) 
            self.absoluteAddress = writeAddress
        elif mode == 'direct':
            self.startAddress = writeAddress  + offset
            self.endAddress = self.startAddress + (n//self.width)
            self.absoluteAddress = writeAddress
        self.aligned = 1 if ((writeAddress + offset) % (n//self.width)) ==0 else 0
        
        print(f"Write : Provided Absolute Address= {writeAddress} , Start Address = {self.startAddress}, End Address ={self.endAddress-1}, Aligned={self.aligned}, offset ={offset}")
        

        self.buffer =  registerRISC(data=data, endian=endian
                                    , n=n
                                    , width=width ### xxxx I added this to solve writing to 32 bits large values
                                    #, listBitsWidth=self.width
                                   )
        self.writeToMemory (data = data, endian=endian)

        
        

        if align==True:
            dataSelected=data
        else:
            dataSelected=[data[cellOrder]]
        self.buffer =  registerRISC(data=dataSelected
                                    , endian=endian
                                    , n=n if align==True else self.width
                                    , width=self.width)
        
        #return 32 bit hex val
        return self.buffer

    #Writes from memory based on start end address calculated from function writeMemoryPtrBuffer
    def writeToMemory(self, data,endian='big'):
        print(f"Write Endian={endian} in {self.idx} | {self.startAddress}->{self.endAddress} | {data} will overwrite {self.Mem[self.startAddress:self.endAddress ]}")
        if endian=='big':
            self.Mem [ self.startAddress : self.endAddress ] = data 
        else:
            self.Mem [ self.startAddress : self.endAddress ][::-1] = data 
        pass

    #calculates the read start/end address then passes control to function readFromMemory
    def readMemoryPtrBuffer(self,ReadAddress, n=32
                            ,endian='big'
                            ,align=True
                            ,cellOrder=0
                           ,mode='indirect'
                            ,offset=0
                           ): ###Test Calculates Start and end address
        #read 32-bit words aligned by 32-bit/8-width  = 4 by default for 32-bit processors
        #To read unaligned data, use align=False, so the read address will be the then specify the amount of bits as n, 
        #self.orderMemory = ReadAddress
        
        #self.startAddress = ReadAddress // (4)  * 4  
        if mode == 'indirect':
            self.startAddress = (ReadAddress + offset) // (n//self.width)  * (n//self.width)    
            #self.orderAligned = ReadAddress
            #self.endAddress = self.startAddress + (4) 
            self.endAddress = self.startAddress + (n//self.width) 
            self.absoluteAddress = ReadAddress
        elif mode == 'direct':
            self.startAddress = (ReadAddress + offset)   
            self.endAddress = self.startAddress + (n//self.width)
            self.absoluteAddress = ReadAddress
            self.absoluteAddressOffset = offset
            self.absoluteAddressOffseted = ReadAddress + offset
        self.aligned = 1 if ((ReadAddress + offset) % (n//self.width)) ==0 else 0
        
        if ReadAddress > self.addressMax or ReadAddress + offset > self.addressMax or ReadAddress + offset < 0 :
            print(f"Error Accessing Location Outside Memory - Padding will be returned")
            #return None
        print(f"Read : Provided Absolute Address= {ReadAddress} , Start Address = {self.startAddress}, End Address ={self.endAddress-1}, Aligned={self.aligned}, offset = {offset}")
        data=self.readFromMemory(endian=endian)
        #print(f"data Accessed = {data}")
        if align==True:
            dataSelected=data
        else:
            dataSelected=[data[cellOrder]]
        #print(f"align={align}, dataSelected = {dataSelected}")
        self.buffer =  registerRISC(data=dataSelected
                                    #, n=self.width
                                    #, listBitsWidth=self.listBitsWidth)
                                    , endian=endian
                                    , n=n if align==True else self.width
                                    , width=self.width)
        #self.convertForms()
        
        #return 32 bit hex val
        return self.buffer



In [1333]:



# In[7]:


class registerRISC(register): #Special class of register, inherits from register class
    def __init__(self, data=0,   **kwargs):
        
        #Width is used when the register loads arrays of 32-bits, we need to specify explicitly their length in advance
        self.width = kwargs.get("width",8) #dta is read in arrays of 1x32-bit for instr and 4x8-bit for data
        
        #This is the cell width for the register, assumed to be 1 byte per cell by default
        #Mainly used in big/little endian forms just in case the register needs to save data to Half Byte or Full word memories
        self.memoryAddressable = kwargs.get("memoryAddressable",8)
        
        #RISC-V is assumed so all registers are operations are done on 32-bit registers or 32-bit aligned memory that is byteaddressable 
        self.n = kwargs.get("n",32)
        
        super().__init__(data=data, n=self.n, endian='big', debug=False
                         , width = self.width
                         , memoryAddressable = self.memoryAddressable
                         , signed=True
                         , Hex=False
                        )


-------------------------------

In [1334]:

class State(object):
    def __init__(self):
        self.IF = {"nop": False, "PC": 0}
        self.ID = {"nop": False, "Instr": 0}
        self.EX = {"nop": False, "Read_data1": 0, "Read_data2": 0, "Imm": 0, "Rs": 0, "Rt": 0, "Wrt_reg_addr": 0, "is_I_type": False, "rd_mem": 0, 
                   "wrt_mem": 0, "alu_op": 0, "wrt_enable": 0}
        self.MEM = {"nop": False, "ALUresult": 0, "Store_data": 0, "Rs": 0, "Rt": 0, "Wrt_reg_addr": 0, "rd_mem": 0, 
                   "wrt_mem": 0, "wrt_enable": 0}
        self.WB = {"nop": False, "Wrt_data": 0, "Rs": 0, "Rt": 0, "Wrt_reg_addr": 0, "wrt_enable": 0}

    def __str__(self):
        return f"{vars(self)}"


# In[4]:


class Core(object):
    def __init__(self, ioDir, imem, dmem, RegOutFilePrefix):
        self.myRF = RegisterFile(ioDir, RegOutFilePrefix)
        self.cycle = 0
        self.halted = False
        self.ioDir = ioDir
        self.state = State()
        self.nextState = State()
        self.ext_imem = imem
        self.ext_dmem = dmem
        
        self.nInstructions = 0
    def __str__(self):
        return f"{vars(self)}"



In [1335]:

# In[87]:


class SingleStageCore(Core):
    def __init__(self, ioDir, imem, dmem, RegOutFilePrefix):
        super(SingleStageCore, self).__init__(os.path.join(ioDir ), imem, dmem, RegOutFilePrefix)
        self.opFilePath = os.path.join(ioDir ,"StateResult_SS.txt") ##Test, Change to \\StateResult_SS.txt
        
    def step(self):
        # Your implementation
        

        
        print( f"\n\n Begin Cycle = {self.cycle}\n\t Before Execution : halted={self.halted}, current={self.state.IF}, Next={self.nextState.IF}")
        
        if not self.nextState.IF["nop"] :
            self.nextState.IF['PC'] = self.state.IF['PC'] + 4
        else:
            self.nextState.IF['PC'] = self.state.IF['PC'] - 4
            
        print( f"\n\n  Cycle ater state transfer = {self.cycle}\n\t Before Execution : halted={self.halted}, current={self.state.IF}, Next={self.nextState.IF}")
        if not self.state.IF["nop"] :
            instruction=imem.readMemoryPtrBuffer(self.state.IF['PC']) #imem is instruction memory
            print (f"instruction in Hex = {instruction.dataHex}")
            print (f"instruction in Bits = {instruction.dataBin}")
            self.instruction=instructionRegister(data=instruction.dataBinMemoryBytesBig
                                                 , PC=self.nextState.IF['PC'] 
                                                 , r=ssCore.myRF  #r is the Registers accessed with same memory class interfaces
                                                 , d=dmem_ss  #d is the data memory
                                                , halted = self.state.IF['nop'] 
                                                )
            self.nInstructions = self.nInstructions + 1                                    
            print( f"\t After Execution : Instruction Variables : PC={self.instruction.PC}, halted = {self.instruction.halted}")

        #if self.nextState.IF["PC"] != self.instruction.PC : self.nextState.IF["PC"] = self.instruction.PC

        print (f'Halt instruction = {self.instruction.halted} \nState NOP =  {self.state.IF["nop"]} ')
        print (f'NextState NOP = {self.nextState.IF["nop"]}')
        print (f'NextState PC = {self.nextState.IF["PC"]}')
        
        self.nextState.IF["PC"] = self.instruction.PC
        self.nextState.IF["nop"] = self.instruction.halted
        print (f'Instruction Halt is loaded into next State Nop\nNextState Nop changed to {self.nextState.IF["nop"]}')

        #self.state.IF['PC'] = self.instruction.PC
        print( f"After Execution : State={self.state.IF}, Next={self.nextState.IF}")        #self.instruction.printInstr()
        if self.state.IF["nop"]:
            self.halted = True

        #Next State Transition
        self.state.IF['PC'] = self.nextState.IF['PC'] #The end of the cycle and updates the current state with the values calculated in this cycle
        self.state.IF['nop'] = self.nextState.IF['nop']

            
        print (f"End of Cycle {self.cycle},Captured Next State = {self.nextState.IF}")
        self.myRF.outputRF(self.cycle) # dump RF
        self.printState(self.nextState, self.cycle) # print states after executing cycle 0, cycle 1, cycle 2 ... 

        print( f"\nEnd Cycle = {self.cycle}\n\t - halted={self.halted}, current={self.state.IF}, Next={self.nextState.IF}")
        self.cycle += 1

    def printState(self, state, cycle):
        printstate = ["-"*70+"\n", "State after executing cycle: " + str(cycle) + "\n"]
        printstate.append("IF.PC: " + str(state.IF["PC"]) + "\n")
        printstate.append("IF.nop: " + str(state.IF["nop"]) + "\n")
        
        if(cycle == 0): perm = "w"
        else: perm = "a"
        with open(self.opFilePath, perm) as wf:
            wf.writelines(printstate)
    
    def __str__(self):
        return f"{vars(self)}"



In [1336]:



# In[15]:


class instruction(register ): 
    def __init__(self, data, PC, halted , r, d):
        super().__init__(data=0, n=32, endian='big', debug=False, memoryAddressable=8, signed=True, hex=False)
        
        self.opcode=self.bit(0,6)
        
        if self.opcode == '0110011':
            self.format='R'
            self.RDecode(r,d)
        elif self.opcode == '0010011' or self.opcode=='0000011':
            self.format='I'
            self.IDecode(r,d)
        elif self.opcode == '1101111':
            self.format='J'
            self.JDecode(r,d)
        elif self.opcode == '1100011':
            self.format='B'
            self.BDecode(r,d)
        elif self.opcode == '0000011':
            self.format='I'
            self.IDecode(r,d)
        elif self.opcode == '0100011':
            self.format='S'
            self.SDecode(r,d)
        elif self.opcode == '1111111':
            self.format='-'
            self.op='HALT'
            self.halted = True
            self.PC = self.PC - 4
            print (f"Instructions Halted, Halted = {self.halted}, emulated by x0+x0=x0")
            self.instrStr=f"format = {self.format}, opname={self.op}, opcode={self.opcode}"
            self.rdInt = 0
            self.rs1Int = 0
            self.rs1R = r.readMemoryPtrBuffer(self.rs1Int) if self.rs1Int !=0 else registerRISC(0)
            self.rs2Int = 0
            self.rs2R = r.readMemoryPtrBuffer(self.rs2Int) if self.rs2Int !=0 else registerRISC(0)
            self.R_ADD(  r, d)

        print (f"Instruction String = {self.instrStr}")
        self.clearx0(r)
        
    def RDecode(self, r, d):
        self.func3=self.bit(12,14)
        
        self.rd=self.bit(7,11)
        self.rdInt=int(self.rd,2)
        
        self.rs1=self.bit(15,19)
        self.rs1Int=int(self.rs1,2)
        self.rs1R = r.readMemoryPtrBuffer(self.rs1Int) if self.rs1Int !=0 else registerRISC(0)
        
        self.rs2=self.bit(20,24)
        self.rs2Int=int(self.rs2,2)
        self.rs2R = r.readMemoryPtrBuffer(self.rs2Int) if self.rs2Int !=0 else registerRISC(0)
        
        self.func7=self.bit(25,31)
        
        if self.func3 =='000' and self.func7=='0000000':
            self.op = 'ADD'
            self.execute = self.R_ADD(r,d)
        elif self.func3=='000' and self.func7=='0100000':
            self.op = 'SUB'
            self.execute = self.R_SUB(r,d) 
        elif self.func3=='100' and self.func7=='0000000':
            self.op = 'XOR'
            self.execute = self.R_XOR(r,d)
        elif self.func3=='110' and self.func7=='0000000':
            self.op = 'OR'
            self.execute = self.R_OR(r,d) 
        elif self.func3=='111' and self.func7=='0000000':
            self.op = 'AND'
            self.execute = self.R_AND(r,d) #NOT TESTED
         
        self.instrStr= f"format = {self.format}, opname={self.op}, opcode={self.opcode},rs1={self.rs1}, rs2={self.rs2}, rd={self.rd}, func3={self.func3}, func7={self.func7}"
        self.execute
        
        
    def IDecode(self ,  r, d):
        self.rd=self.bit(7,11)
        self.rdInt=int(self.rd,2)

        self.func3=self.bit(12,14)
        
        self.rs1=self.bit(15,19)
        self.rs1Int=int(self.rs1,2)
        self.rs1R = r.readMemoryPtrBuffer(self.rs1Int) if self.rs1Int !=0 else registerRISC(0)

        
        self.imm=self.bit(20,31)
        #self.immInt=int(self.imm,2)
        #self.immR=registerRISC(data=self.imm)
        self.immR=registerRISC(register(self.imm,signed=True).dataInt)
        self.immInt=self.immR.dataInt
        
        if self.func3=='000' and self.opcode=='0010011':
            self.op = 'ADDI'
            self.execute = self.I_ADDI(r,d)
        elif self.func3=='100' :
            self.op = 'XORI'
            self.execute = self.I_XORI(r,d)
        elif self.func3=='110' :
            self.op = 'ORI'
            self.execute = self.I_ORI(r,d)
        elif self.func3=='111':
            self.op = 'ANDI'
            self.execute = self.I_ANDI(r,d)
        elif self.func3=='000' and self.opcode=='0000011':
            self.op = 'LW'
            self.execute = self.I_LW(r,d)
            
        self.instrStr= f"format = {self.format}, opname={self.op}, opcode={self.opcode},rs1={self.rs1},  rd={self.rd}, func3={self.func3}, imm={self.imm}"
        self.execute
        #self.clearx0(r)

    def SDecode(self ,  r, d):
        self.imm=self.bit(25,31) + self.bit(7,11)
        #self.immInt=int(self.imm,2)
        #self.immR=registerRISC(data=self.imm)
        self.immR=registerRISC(register(self.imm,signed=True).dataInt)
        self.immInt=self.immR.dataInt
        
        self.func3=self.bit(12,14)
        
        self.rs1=self.bit(15,19)
        self.rs1Int=int(self.rs1,2)
        self.rs1R = r.readMemoryPtrBuffer(self.rs1Int) if self.rs1Int !=0 else registerRISC(0)

        self.rs2=self.bit(20,24)
        self.rs2Int=int(self.rs2,2)
        self.rs2R = r.readMemoryPtrBuffer(self.rs2Int) if self.rs2Int !=0 else registerRISC(0)
        
        if self.func3=='010':
            self.op = 'SW'
            self.execute = self.S_SW(r,d)

        self.instrStr= f"format = {self.format}, opname={self.op}, opcode={self.opcode},rs1={self.rs1}, rs2={self.rs2}, func3={self.func3},  imm={self.imm}"
        self.execute
        
    def BDecode(self ,  r, d):
        #self.imm= self.bit(7)+self.bit(7)+self.bit(25,30)+self.bit(8,11)+'0'
        self.imm= self.bit(31)+self.bit(7)+self.bit(25,30)+self.bit(8,11)+'0'
        #self.immInt=int(self.imm,2)
        #elf.immR=registerRISC(data=self.imm)
        self.immR=registerRISC(register(self.imm,signed=True).dataInt)
        self.immInt=self.immR.dataInt

        self.func3=self.bit(12,14)

        self.rs1=self.bit(15,19)
        self.rs1Int=int(self.rs1,2)
        self.rs1R = r.readMemoryPtrBuffer(self.rs1Int) if self.rs1Int !=0 else registerRISC(0)

        self.rs2=self.bit(20,24)
        self.rs2Int=int(self.rs2,2)
        self.rs2R = r.readMemoryPtrBuffer(self.rs2Int) if self.rs2Int !=0 else registerRISC(0)

        if self.func3=='000':
            self.op = 'BEQ'
            self.execute = self.B_BEQ(r,d) #NOT TESTED
        elif self.func3=='001' :
            self.op = 'BNE'
            self.execute = self.B_BNE(r,d)
        self.instrStr= f"format = {self.format}, opname={self.op}, opcode={self.opcode},rs1={self.rs1}, rs2={self.rs2}, func3={self.func3},  imm={self.imm}"

        self.execute
        
    def UDecode(self ,  r, d):
        self.rd=self.bit(7,11)
        self.rdInt=int(self.rd,2)

        self.imm=self.bit(12,31)+'0'*12 #12 + 31-12 + 1
        #self.immInt=int(self.imm,2)
        #elf.immR=registerRISC(data=self.imm)
        self.immR=registerRISC(register(self.imm,signed=True).dataInt)
        self.immInt=self.immR.dataInt
        
        self.instrStr= f"format = {self.format}, opname={self.op}, opcode={self.opcode}, rd={self.rd},  imm={self.imm}"
        
        #self.clearx0(r)
        
    def JDecode(self ,  r, d):
        self.rd=self.bit(7,11)
        self.rdInt=int(self.rd,2)
        
        #self.imm= self.bit(31)+self.bit(12,12+8)+self.bit(12+8+1)+self.bit(12+8+1 + 1, 12+8+1 + 1 + 10)+'0'
        #self.imm= self.bit(31)+self.bit(10,18)+self.bit(19)+self.bit(20,30)+'0' #1+10+1+1+8+1+1
        self.imm= self.bit(31)+self.bit(12,19)+self.bit(20)+self.bit(21,30)+'0' #1+10+1+1+8+1+1
        #self.immInt=int(self.imm,2)
        #self.immR=registerRISC(data=self.imm)
        self.immR=registerRISC(register(self.imm,signed=True).dataInt)
        self.immInt=self.immR.dataInt
        
        self.op='JAL'
        self.instrStr= f"format = {self.format}, opname={self.op}, opcode={self.opcode}, rd={self.rd},  imm={self.imm}"
        self.execute = self.J_JAL(r,d)
        self.execute
        #self.clearx0(r)
        
    def printInstr(self):
        #print(f"format = {self.format}, opcode={self.opcode},rs1={self.rs1}, rs2={self.rs2}, rd={self.rd}, func3={self.func3}, func7={self.func7}, imm={self.imm}")

        if self.format == 'R':
            print(f"format = {self.format}, opname={self.op}, opcode={self.opcode},rs1={self.rs1}, rs2={self.rs2}, rd={self.rd}, func3={self.func3}, func7={self.func7}")
        elif self.format == 'I':
            print(f"format = {self.format}, opname={self.op}, opcode={self.opcode},rs1={self.rs1},  rd={self.rd}, func3={self.func3}, imm={self.imm}")
        elif self.format == 'S':
            print(f"format = {self.format}, opname={self.op}, opcode={self.opcode},rs1={self.rs1}, rs2={self.rs2}, func3={self.func3},  imm={self.imm}")
        elif self.format == 'B':
            print(f"format = {self.format}, opname={self.op}, opcode={self.opcode},rs1={self.rs1}, rs2={self.rs2}, func3={self.func3},  imm={self.imm}")
        elif self.format == 'U':
            print(f"format = {self.format}, opname={self.op}, opcode={self.opcode}, rd={self.rd},  imm={self.imm}")
        elif self.format == 'J':
            print(f"format = {self.format}, opname={self.op}, opcode={self.opcode}, rd={self.rd},  imm={self.imm}")
        elif self.format == '-':
            print(f"format = {self.format}, opname={self.op}, opcode={self.opcode}")

    def S_SW(self ,  r, d):
        #print(f"rs2 {self.rs2R}")
        print(f"\t op={self.op} rs2={self.rs2Int}, imm={self.immInt}(rs1={self.rs1Int})")
        #self.temp = d.readMemoryPtrBuffer(self.rs2R.data)
        self.temp = r.readMemoryPtrBuffer(self.rs2Int )#, width=32)
        print(f"\t M[rs2]=  {self.temp.dataHexMemoryBytesBig} = {self.temp.dataBin} = {self.temp.dataInt} ")
        print(f"\t temp=  {[self.temp.dataBinMemoryBytesBig]} ")
        d.writeMemoryPtrBuffer(data=self.temp.dataBinMemoryBytesBig, writeAddress=self.rs1Int,  offset=self.immInt)

    def I_LW(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, imm={self.immInt}(r1={self.rs1Int})")
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        self.temp = d.readMemoryPtrBuffer(self.rs1Int, offset=self.immInt)
        print(f"\t M[rs1 + imm]=  {self.temp.dataHexMemoryBytesBig} = {self.temp.dataBin} = {self.temp.dataInt} ")
        print(f"\t temp=  {[self.temp.dataBin]} ")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)


        
    def R_ADD(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, r1={self.rs1Int} , r2={self.rs2Int}")
        self.temp = self.rs1R + self.rs2R
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        print(f"\t r1={self.rs1R.dataHexMemoryBytesBig} +\n\t r2={self.rs2R.dataHexMemoryBytesBig} =\n\t rd={self.temp.dataHexMemoryBytesBig} ")
        #print(f"\t temp=  {[self.temp.dataBin]} ")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)

    def R_SUB(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, r1={self.rs1Int} , r2={self.rs2Int}")
        self.temp = self.rs1R - self.rs2R
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        print(f"\t r1={self.rs1R.dataHexMemoryBytesBig} +\n\t r2={self.rs2R.dataHexMemoryBytesBig} =\n\t rd={self.temp.dataHexMemoryBytesBig} ")
        #print(f"\t temp=  {[self.temp.dataBin]} ")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)

        #NOT Tested
    def R_AND(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, r1={self.rs1Int} , r2={self.rs2Int}")
        self.temp = self.rs1R & self.rs2R
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        print(f"\t r1={self.rs1R.dataHexMemoryBytesBig} +\n\t r2={self.rs2R.dataHexMemoryBytesBig} =\n\t rd={self.temp.dataHexMemoryBytesBig} ")
        #print(f"\t temp=  {[self.temp.dataBin]} ")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)

    def R_OR(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, r1={self.rs1Int} , r2={self.rs2Int}")
        self.temp = self.rs1R | self.rs2R
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        print(f"\t r1={self.rs1R.dataHexMemoryBytesBig} +\n\t r2={self.rs2R.dataHexMemoryBytesBig} =\n\t rd={self.temp.dataHexMemoryBytesBig} ")
        #print(f"\t temp=  {[self.temp.dataBin]} ")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)

    def R_XOR(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, r1={self.rs1Int} , r2={self.rs2Int}")
        self.temp = self.rs1R ^ self.rs2R
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        print(f"\t r1={self.rs1R.dataHexMemoryBytesBig} +\n\t r2={self.rs2R.dataHexMemoryBytesBig} =\n\t rd={self.temp.dataHexMemoryBytesBig} ")
        #print(f"\t temp=  {[self.temp.dataBin]} ")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)

        
    def clearx0(self, r):
        print("\n Clearing x0")
        r.writeMemoryPtrBuffer(data=['0'*32], writeAddress=0,  offset=0, width=32)

    def I_ADDI(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, imm={self.immInt}(r1={self.rs1Int})")
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        #self.temp = d.readMemoryPtrBuffer(self.rs1Int, offset=0) + registerRISC(self.immInt)
        #self.temp = d.readMemoryPtrBuffer(self.rs1Int, offset=0) + self.immR.dataInt
        self.temp = self.rs1R  + self.immR
        print(f"\t imm=\t{self.immR.dataBin} = {self.immR.dataInt}")
        print(f"\t +R1=\t{self.rs1R.dataBin} = {self.rs1R.dataInt}")
        print(f"\t temp=\t{self.temp.dataBin} = {self.temp.dataInt}")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)
        

    def I_ORI(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, imm={self.immInt}(r1={self.rs1Int})")
        #print(f"\t op={self.op} rd={self.rdInt}, imm={self.immInt}(r1={self.rs1Int})")
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        #self.temp = d.readMemoryPtrBuffer(self.rs1Int, offset=0) + registerRISC(self.immInt)
        #self.temp = d.readMemoryPtrBuffer(self.rs1Int, offset=0) + self.immR.dataInt
        self.temp = self.rs1R  | self.immR
        print(f"\t imm=\t{self.immR.dataBin} = {self.immR.dataInt}")
        print(f"\t +R1=\t{self.rs1R.dataBin} = {self.rs1R.dataInt}")
        print(f"\t temp=\t{self.temp.dataBin} = {self.temp.dataInt}")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)
        
    def I_ANDI(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, imm={self.immInt}(r1={self.rs1Int})")
        #print(f"\t op={self.op} rd={self.rdInt}, imm={self.immInt}(r1={self.rs1Int})")
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        #self.temp = d.readMemoryPtrBuffer(self.rs1Int, offset=0) + registerRISC(self.immInt)
        #self.temp = d.readMemoryPtrBuffer(self.rs1Int, offset=0) + self.immR.dataInt
        self.temp = self.rs1R  & self.immR
        print(f"\t imm=\t{self.immR.dataBin} = {self.immR.dataInt}")
        print(f"\t +R1=\t{self.rs1R.dataBin} = {self.rs1R.dataInt}")
        print(f"\t temp=\t{self.temp.dataBin} = {self.temp.dataInt}")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)
        
    def I_XORI(self ,  r, d):
        print(f"\t op={self.op} rd={self.rdInt}, imm={self.immInt}(r1={self.rs1Int})")
        #print(f"\t op={self.op} rd={self.rdInt}, imm={self.immInt}(r1={self.rs1Int})")
        #self.temp = d.readMemoryPtrBuffer(self.rs1R.data, offset=self.immInt)
        #self.temp = d.readMemoryPtrBuffer(self.rs1Int, offset=0) + registerRISC(self.immInt)
        #self.temp = d.readMemoryPtrBuffer(self.rs1Int, offset=0) + self.immR.dataInt
        self.temp = self.rs1R  ^ self.immR
        print(f"\t imm=\t{self.immR.dataBin} = {self.immR.dataInt}")
        print(f"\t +R1=\t{self.rs1R.dataBin} = {self.rs1R.dataInt}")
        print(f"\t temp=\t{self.temp.dataBin} = {self.temp.dataInt}")
        r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)
        
        #print(f"\t rs1 + imm =  {self.temp.dataHexMemoryBytesBig} = {self.temp.dataBin} = {self.temp.dataInt} ")
        #print(f"\t temp=  {[self.temp.dataBin]} ")
        #r.writeMemoryPtrBuffer(data=[self.temp.dataBin], writeAddress=self.rdInt,  offset=0, width=32)

    def B_BNE(self, r,d):
        print(f"\t op={self.op} rs1={self.rs1R.dataBin} rs2 ={self.rs2R.dataBin}, imm={self.immR.dataInt} ")
        print(f"\t Is rs1={self.rs1R.dataBin} NEq to rs2 ={self.rs2R.dataBin} ? {self.rs1R != self.rs2R}")
        if self.rs1R != self.rs2R :
            print(f"\t Not Equal : Branch Taken, PC will change from {self.PC} by offset {self.immR.dataInt} to be = {self.PC + self.immR.dataInt}")
            self.PC = self.PC - 4 + self.immR.dataInt #self.PC is the next state PC+4 
        else:
            print(f"\t Equal : Branch Not Taken, PC will remain from {self.PC}, ignoring the offset {self.immR.dataInt}")

    def B_BEQ(self, r,d): #NOT TESTED
        print(f"\t op={self.op} rs1={self.rs1R.dataBin} rs2 ={self.rs2R.dataBin}, imm={self.immR.dataInt} ")
        print(f"\t Is rs1={self.rs1R.dataBin} Eq to rs2 ={self.rs2R.dataBin} ? {self.rs1R == self.rs2R}")
        if self.rs1R == self.rs2R :
            print(f"\t Equal : Branch Taken, PC will change from {self.PC} by offset {self.immR.dataInt} to be = {self.PC + self.immR.dataInt}")
            self.PC = self.PC - 4 + self.immR.dataInt
        else:
            print(f"\t Not Equal : Branch Not Taken, PC will remain from {self.PC}, ignoring the offset {self.immR.dataInt}")

    def J_JAL(self, r,d): #Assumes input is PC+4
        #self.rdInt=int(self.rd,2)
        #self.immInt=self.immR.dataInt
        #self.temp.dataBin = self.immR.dataInt
        print(f"\t op={self.op} rd={self.rdInt}, imm={self.immInt}")
        print(f"\t Current Next State PC ={self.PC} will be saved in rd={self.rdInt}")
        print(f"\t Then the Current State PC = {self.PC - 4} will jump by offset {self.immR.dataInt} to {self.PC - 4 + self.immR.dataInt}")
        temp = registerRISC(self.PC)
        r.writeMemoryPtrBuffer(data=temp.dataBin ,writeAddress=self.rdInt,  offset=0, width=32)
        self.PC = self.PC - 4 + self.immR.dataInt



In [1337]:
'''
class regIFID(register):
    self.pc = 0
    #self.instruction = 
    
    class instructionRegister(register ): #Class used to decode and execute the instructions
    #We provide PC+4 to the instruction, that is why there is a -4 for jump instructions
    #There instructionRegister will generate Halted and PC attributes that will be fed to the next state object
    def __init__(self, data, PC, halted , r, d):
        super().__init__(data=data, n=32, endian='big', debug=False, memoryAddressable=8, signed=True, hex=False)

'''

"\nclass regIFID(register):\n    self.pc = 0\n    #self.instruction = \n    \n    class instructionRegister(register ): #Class used to decode and execute the instructions\n    #We provide PC+4 to the instruction, that is why there is a -4 for jump instructions\n    #There instructionRegister will generate Halted and PC attributes that will be fed to the next state object\n    def __init__(self, data, PC, halted , r, d):\n        super().__init__(data=data, n=32, endian='big', debug=False, memoryAddressable=8, signed=True, hex=False)\n\n"

In [1338]:
class stageFetch():
    def __init__():
        self.pc = 0

---------------------------

In [1339]:

# In[5]:


class FiveStageCore(Core):
    def __init__(self, ioDir, imem, dmem, RegOutFilePrefix):
        super(FiveStageCore, self).__init__(os.path.join(ioDir), imem, dmem, RegOutFilePrefix)
        self.opFilePath = os.path.join(ioDir ,"StateResult_FS.txt") ###Test, change to \\StateResult_FS.txt

    def step(self):
        # Your implementation
        # --------------------- WB stage ---------------------
        
        
        
        # --------------------- MEM stage --------------------
        
        
        
        # --------------------- EX stage ---------------------
        
        
        
        # --------------------- ID stage ---------------------
        
        
        
        # --------------------- IF stage ---------------------
        
        self.halted = True
        if self.state.IF["nop"] and self.state.ID["nop"] and self.state.EX["nop"] and self.state.MEM["nop"] and self.state.WB["nop"]:
            self.halted = True
        
        self.myRF.outputRF(self.cycle) # dump RF
        self.printState(self.nextState, self.cycle) # print states after executing cycle 0, cycle 1, cycle 2 ... 
        
        self.state = self.nextState #The end of the cycle and updates the current state with the values calculated in this cycle
        self.cycle += 1

    def printState(self, state, cycle):
        printstate = ["-"*70+"\n", "State after executing cycle: " + str(cycle) + "\n"]
        printstate.extend(["IF." + key + ": " + str(val) + "\n" for key, val in state.IF.items()])
        printstate.extend(["ID." + key + ": " + str(val) + "\n" for key, val in state.ID.items()])
        printstate.extend(["EX." + key + ": " + str(val) + "\n" for key, val in state.EX.items()])
        printstate.extend(["MEM." + key + ": " + str(val) + "\n" for key, val in state.MEM.items()])
        printstate.extend(["WB." + key + ": " + str(val) + "\n" for key, val in state.WB.items()])

        if(cycle == 0): perm = "w"
        else: perm = "a"
        with open(self.opFilePath, perm) as wf:
            wf.writelines(printstate)



In [1340]:

class RegisterFile(Memory):
    def __init__(self, ioDir, RegOutFilePrefix):
        #self.outputFile = ioDir + "RFResult.txt"
        self.outputFile = os.path.join(ioDir , RegOutFilePrefix + "RFResult.txt")
        self.Registers = [0x0 for i in range(32)]
    
    

        Memory.__init__(self, name=RegOutFilePrefix+'RFResult'
                 ,ioDir=ioDir
                 , inputFile = False
                ,outputFile=self.outputFile
                 ,nCellsMax=32
                 , extend=True
                 , width = 32 
                  ,signed = True #True to be able to access both signed and unsigned versions
                    ,data=['0'*32]*32 #xxx I added this line to initialize the registers
                )


    def readRF(self, Reg_addr):
        # Fill in
        #I used my function readMemoryPtrBuffer , for example r.readMemoryPtrBuffer(self.rs2Int ) is the register number
        # for data memory I used the alias d, and for registery alias r.
 
        pass
    
    def writeRF(self, Reg_addr, Wrt_reg_data):
        # Fill in
        # I used my function writeMemoryPtrBuffer , for example to read from memory address 0 with offset=4 as an array of bits
        # d.writeMemoryPtrBuffer(data=self.temp.dataBinMemoryBytesBig, writeAddress=0,  offset=4)
        pass
         
    def outputRF(self, cycle):
        print (f"Updating Regisry Stats at {self.outputFile} for cycle {cycle}")
        op = ["-"*70+"\n", "State of RF after executing cycle:" + str(cycle) + "\n"]
        #op.extend([str(val)+"\n" for val in self.Registers]) I added the below line yyy
        op.extend([str(val)+"\n" for val in self.Mem])
        if(cycle == 0): perm = "w"
        else: perm = "a"
        with open(self.outputFile, perm) as file:
            file.writelines(op)

            #xxxx

In [1341]:

# In[14]:


class InsMem( Memory):
    def __init__(self, name, ioDir):
        self.id = name
        self.inputFileName = "imem.txt"
        #self.inputFile = ioDir + "\\" + self.inputFileName
        self.inputFile = os.path.join(ioDir ,self.inputFileName)
        with open(self.inputFile) as im:
            self.IMem = [data.replace("\n", "") for data in im.readlines()]
        
        
        Memory.__init__(self, name=self.id
              ,ioDir=ioDir, inputFileName=self.inputFileName
              , extend=False
             )
        
        
    def readInstr(self, ReadAddress):
        #read instruction memory
        #
        
        pass
          
class DataMem(Memory):
    def __init__(self, name, ioDir):
        self.id = name
        self.ioDir = ioDir
        self.resPath = os.path.join(self.ioDir , self.id + "_DMEMResult.txt")
        self.inputFileName = "dmem.txt"
        self.inputFile = os.path.join(ioDir , self.inputFileName)
        with open(self.inputFile) as dm:
            self.DMem = [data.replace("\n", "") for data in dm.readlines()]


        Memory.__init__(self, name=self.id
                     ,ioDir=ioDir
                     ,inputFileName=self.inputFileName
                     ,outputFile=self.resPath
                     ,nCellsMax=1000
                     , extend=True #xxx I added the extension to fill the empty parts of the data memory
                    )



    def readInstr(self, ReadAddress):
        #read data memory
        #return 32 bit hex val
        #Iused my own function with alis imem ; imem.readMemoryPtrBuffer(self.state.IF['PC'])
        pass
        
    def writeDataMem(self, Address, WriteData):
        # write data into byte addressable memory,
        # I used my own function; d.writeMemoryPtrBuffer(data=self.temp.dataBinMemoryBytesBig, writeAddress=self.rs1Int,  offset=self.immInt)
        pass
                     
    def outputDataMem(self):
        #resPath = self.ioDir + "\\" + self.id + "_DMEMResult.txt"
        with open(self.resPath, "w") as rp:
            #rp.writelines([str(data) + "\n" for data in self.DMem])
            rp.writelines([str(data) + "\n" for data in self.Mem]) # I added Mem instead of DMem yyy


In [1342]:

# In[40]:


if __name__ == "__main__":
     
    #parse arguments for input file location
    parser = argparse.ArgumentParser(description='RV32I processor')
    parser.add_argument('--iodir', default="", type=str, help='Directory containing the input files.')
    
    ### parse the command arguments
    #args = parser.parse_args() #Test, replace with args = parser.parse_args()
    #args = parser.parse_args(args=['--iodir','C:\\Users\\life_\\Projects\\RISC_Simulator\\RISC-V_Simulator_Repo\\sample_testcases_S24\\input\\testcase1']) #Test, replace with args = parser.parse_args()
    args = parser.parse_args(args=[ '--iodir',
        '/usr/local/google/home/mhafez/Downloads/Simulator/sample_testcases_S24/input/testcase1/'
    ]
                            )
    #Test, replace with args = parser.parse_args()
    #/usr/local/google/home/mhafez/Downloads/Simulator/sample_testcases_S24/input/testcase1/Code.asm
    ioDir = os.path.abspath(args.iodir)
    print("IO Directory:", ioDir)

    imem = InsMem("Imem", ioDir)
    dmem_ss = DataMem("SS", ioDir)
    dmem_fs = DataMem("FS", ioDir) 
    
    ssCore = SingleStageCore(ioDir, imem, dmem_ss, RegOutFilePrefix='SS_')
    fsCore = FiveStageCore(ioDir, imem, dmem_fs , RegOutFilePrefix='FS_')


    while(True):
        if not ssCore.halted:
            ssCore.step()
        
        if not fsCore.halted:
            fsCore.step()

        if ssCore.halted and fsCore.halted:
            break
    
    
    print (f"Done Executing Instructions")
    print (f"Performance of Single Stage:")
    print (f"#Cycles -> {ssCore.cycle}")
    print (f"#Instructions -> {ssCore.nInstructions}")
    print (f"CPI -> {ssCore.cycle/ssCore.nInstructions}")
    print (f"IPC -> {ssCore.nInstructions/ssCore.cycle}")
    PerformanceMetrics = [f"Performance of Single Stage:\n"
                          ,f"#Cycles -> {ssCore.cycle}\n"
                          ,f"#Instructions -> {ssCore.nInstructions}\n"
                          ,f"CPI -> {ssCore.cycle/ssCore.nInstructions}\n"
                          ,f"IPC -> {ssCore.nInstructions/ssCore.cycle}\n"
                         ]
    with open(os.path.join(ioDir , "PerformanceMetrics.txt"), mode='w') as wf:
        wf.writelines(PerformanceMetrics)
    print ("Done")


    # dump SS and FS data mem.
    dmem_ss.outputDataMem()
    dmem_fs.outputDataMem()


# In[ ]:





# In[ ]:






IO Directory: /usr/local/google/home/mhafez/Downloads/Simulator/sample_testcases_S24/input/testcase1


 Begin Cycle = 0
	 Before Execution : halted=False, current={'nop': False, 'PC': 0}, Next={'nop': False, 'PC': 0}


  Cycle ater state transfer = 0
	 Before Execution : halted=False, current={'nop': False, 'PC': 0}, Next={'nop': False, 'PC': 4}
Read : Provided Absolute Address= 0 , Start Address = 0, End Address =3, Aligned=1, offset = 0
Read Endian=big Imem | 0->3 | ['00000000', '00000000', '00000000', '10000011'] 
instruction in Hex = 00000083
instruction in Bits = 00000000000000000000000010000011
Write : Provided Absolute Address= 0 , Start Address = 0, End Address =0, Aligned=1, offset =0
Write Endian=big in SS_RFResult | 0->1 | ['00000000000000000000000000000000'] will overwrite ['00000000000000000000000000000000']
	 op=LW rd=1, imm=0(r1=0)
Read : Provided Absolute Address= 0 , Start Address = 0, End Address =3, Aligned=1, offset = 0
Read Endian=big SS | 0->3 | ['01010101', '0101

In [1343]:
xmem = InsMem("Imem", ioDir)

In [1344]:
rx = fsCore.myRF
print(rx)

{'outputFile': '/usr/local/google/home/mhafez/Downloads/Simulator/sample_testcases_S24/input/testcase1/FS_RFResult.txt', 'Registers': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'idx': 'FS_RFResult', 'ioDir': '/usr/local/google/home/mhafez/Downloads/Simulator/sample_testcases_S24/input/testcase1', 'memDataFile': False, 'width': 32, 'nCellsMax': 32, 'cycle': 0, 'extend': True, 'inputFileName': False, 'inputFile': False, 'Mem': ['00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '00000000000000000000000000000000', '0000000000

In [1345]:
class instruction(register ): #Class used to decode and execute the instructions
    #We provide PC+4 to the instruction, that is why there is a -4 for jump instructions
    #There instructionRegister will generate Halted and PC attributes that will be fed to the next state object
    def __init__(self, data):
        super().__init__(data=data, n=32, endian='big', debug=False, memoryAddressable=8, signed=True, hex=False)
            
        
        self.opcode=self.bit(0,6)
        
        if self.opcode == '0110011':
            self.format='R'
            #self.RDecode(r,d)
        elif self.opcode == '0010011' or self.opcode=='0000011':
            self.format='I'
            #self.IDecode(r,d)
        elif self.opcode == '1101111':
            self.format='J'
            #self.JDecode(r,d)
        elif self.opcode == '1100011':
            self.format='B'
            #self.BDecode(r,d)
        elif self.opcode == '0000011':
            self.format='I'
            #self.IDecode(r,d)
        elif self.opcode == '0100011':
            self.format='S'
            #self.SDecode(r,d)
        elif self.opcode == '1111111':
            self.format='-'
            self.op='HALT'

            
            
    def __str__(self): #Use the print statement to show all attributes of the register
        return f"{vars(self)}"

        


In [1346]:
'''
x=instruction(0)
print(x.dataBin)

z=imem.readMemoryPtrBuffer(0)
x=instruction(z.dataBin)
x.opcode
'''

'\nx=instruction(0)\nprint(x.dataBin)\n\nz=imem.readMemoryPtrBuffer(0)\nx=instruction(z.dataBin)\nx.opcode\n'

In [1347]:
class FiveStageCore(Core):
    def __init__(self, ioDir, imem, dmem, RegOutFilePrefix):
        super(FiveStageCore, self).__init__(os.path.join(ioDir), imem, dmem, RegOutFilePrefix)
        self.opFilePath = os.path.join(ioDir ,"StateResult_FS.txt") ###Test, change to \\StateResult_FS.txt
        
        #self.ifSignals=ifSignals()
        #self.ifidR=ifidR(self.ifSignals)
        #self.ifDev=ifDev(self.ifSignals)
        self.signals=signals()
        self.registers=registers(self.signals)
        #self.devices=devices(self.signals , self.registers)
        
    def ifStage(self):
        
        
        instructionFetched = instruction(data=imem.readMemoryPtrBuffer(self.signals.ifSignals.pc).dataBin)
        
        self.signals.ifSignals.pc4 = self.signals.ifSignals.pc + 4
        
        
        self.signals.ifSignals.pcSrc = self.signals.ifSignals.ifidControl & self.signals.ifSignals.ifidComparatorOut

        #self.signals.ifSignals.pcn1 = self.devices.ifDev.ifMux1.exe()
        self.signals.ifSignals.pcn1 = {0:self.signals.ifSignals.pc4
                                      ,1:self.signals.ifSignals.pcImm
                                      }.get(self.signals.ifSignals.pcSrc)
        
        
        
        #Transition
        if self.signals.ifSignals.ifidWrite == 1 and self.signals.ifSignals.ifidFlush == 0:
            self.registers.ifidR.instruction = instructionFetched
            self.registers.ifidR.pc = self.signals.ifSignals .pc
            
        elif self.signals.ifSignals.ifidFlush == 1 : #Squash
            self.registers.ifidR.instruction = instruction(data=registerRISC(0))#.dataBin)
            self.registers.ifidR.pc = 0
            
        
        if self.signals.ifSignals.pcWrite == 1:
            self.signals.ifSignals.pc = self.signals.ifSignals.pcn1

            
    def wbStage(self):
        #if self.registers.memwbR.control_wb_memtoReg != 0 and self.registers.memwbR.control_wb_regWrite != 0:
        
        if (self.registers.memwbR.control_wb_memtoReg | self.registers.memwbR.control_wb_regWrite) != 0 :
        
            self.signals.wbSignals.writeData = {0: self.registers.memwbR.aluOut
                                                    ,1: self.registers.memwbR.memOut

                }.get(self.registers.memwbR.control_wb_memtoReg)


            if self.registers.memwbR.control_wb_regWrite == 1 : 
                self.myRF.writeMemoryPtrBuffer(data= self.signals.wbSignals.writeData.dataBinMemoryBytesBig
                                           , writeAddress=self.registers.memwbR.rd
                                           ,  offset=0
                                           , width=32
                                          )
            
    def memStage(self):
        
        if (self.registers.exmemR.control_wb_memtoReg | self.registers.exmemR.control_wb_regWrite |
            self.registers.exmemR.control_mem_memRead | self.registers.exmemR.control_mem_memWrite
           
           ) != 0 :

        
            if self.registers.exmemR.control_mem_memRead == 1 :#and self.registers.exmemR.control_mem_memWrite == 0:
                self.signals.memReadData = self.ext_dmem.readMemoryPtrBuffer( self.registers.exmemR.aluOut.dataInt
                                                                    , offset=0 )

                self.registers.memwbR.memOut = self.signals.memSignals.memReadData

            if self.registers.exmemR.control_mem_memWrite == 1 :#and self.registers.exmemR.control_mem_memWrite == 1:
                self.ext_dmem.writeMemoryPtrBuffer(data= self.registers.exmemR.r2Data.dataBinMemoryBytesBig
                                               , writeAddress=self.registers.exmemR.aluOut.dataInt
                                               ,  offset=0)
            

        self.registers.memwbR.control_wb_regWrite = self.registers.exmemR.control_wb_regWrite
        self.registers.memwbR.control_wb_memtoReg = self.registers.exmemR.control_wb_memtoReg
        
        self.registers.memwbR.aluOut = self.registers.exmemR.aluOut
        self.registers.memwbR.rd = self.registers.exmemR.rd
        self.registers.memwbR.pc = self.registers.exmemR.pc
        
    def exStage(self):
        
        if (self.registers.idexR.control_wb_memtoReg | self.registers.idexR.control_wb_regWrite |
            self.registers.idexR.control_mem_memRead | self.registers.idexR.control_mem_memWrite |
            self.registers.idexR.control_ex_aluSrc |  self.registers.idexR.control_ex_aluOp
           ) != 0 : 
        
            exAluControl.exec(self)
            exForwarder.exec(self)
            muxABC.exec(self)
            exALU.exec(self)
        
        
        self.registers.exmemR.control_wb_memtoReg = self.registers.idexR.control_wb_memtoReg
        self.registers.exmemR.control_wb_regWrite = self.registers.idexR.control_wb_regWrite
        self.registers.exmemR.control_mem_memRead = self.registers.idexR.control_mem_memRead
        self.registers.exmemR.control_mem_memWrite = self.registers.idexR.control_mem_memWrite
        
        self.registers.exmemR.aluOut = self.registers.exmemR.aluOut
        self.registers.exmemR.rd = self.registers.exmemR.rd
        self.registers.exmemR.pc = self.registers.exmemR.pc
        
        
    def __str__(self): #Use the print statement to show all attributes of the register
        return f"{vars(self)}"


In [1348]:
class signals():
    def __init__(self):
        self.ifSignals=ifSignals()
        self.wbSignals=wbSignals()
        self.memSignals=memSignals()
        self.exSignals=exSignals()

        
class registers():
    def __init__(self, signals):
        self.ifidR=ifidR(signals.ifSignals)
        self.memwbR=memwbR(signals.ifSignals)
        self.exmemR=exmemR(signals.ifSignals)
        self.idexR=idexR(signals.ifSignals)
        
class devices():
    def __init__(self, signals, registers):
        #self.exDev=exDev(self, signals, registers)
        self.exDev=exDev(self, signals, registers)
        

In [1362]:
x = FiveStageCore(ioDir, imem, dmem_fs , RegOutFilePrefix='FS_')
x.wbStage()
x.memStage()
x.exStage()

In [1360]:


x.registers.idexR.control_ex_aluOp, \
x.registers.idexR.func3,  \
x.registers.idexR.func7bit30 , \
x.signals.exSignals.aluControlIn

x.registers.idexR.rs1 = 1

x.registers.exmemR.rd = 1
x.registers.exmemR.control_wb_regWrite = 1

x.registers.memwbR.rd = 1
x.registers.memwbR.control_wb_regWrite = 1

x.registers.idexR.control_ex_aluSrc = 1
x.registers.idexR.imm = registerRISC(70)

x.registers.idexR.readData1 = registerRISC(20)
x.registers.idexR.readData2 = registerRISC(40)

x.registers.exmemR.aluOut = registerRISC(10)
x.registers.memwbR.memOut = registerRISC(60)

x.registers.idexR.control_ex_aluSrc = 1

x.signals.exSignals.fwdA, x.signals.exSignals.fwdB ,\
x.signals.exSignals.aluIn1.dataInt, x.signals.exSignals.aluIn2.dataInt,\
x.signals.exSignals.aluOut.dataInt

(0, 0, 0, 0, 0)

In [1350]:
exAluControl.exec(x)
exForwarder.exec(x)
x.signals.exSignals.fwdA, x.signals.exSignals.fwdB ,\
x.signals.exSignals.aluIn1.dataInt, x.signals.exSignals.aluIn2.dataInt,\
x.signals.exSignals.aluOut.dataInt

A : Ex Hazard


(2, 0, 0, 0, 0)

In [1351]:
muxABC.exec(x)
x.signals.exSignals.fwdA, x.signals.exSignals.fwdB ,\
x.signals.exSignals.aluIn1.dataInt, x.signals.exSignals.aluIn2.dataInt,\
x.signals.exSignals.aluOut.dataInt

(2, 0, 10, 70, 0)

In [1352]:
x.signals.exSignals.aluIn1.dataInt

10

In [1354]:
#x.exStage()
exALU.exec(x)
x.signals.exSignals.fwdA, x.signals.exSignals.fwdB ,\
x.signals.exSignals.aluIn1.dataInt, x.signals.exSignals.aluIn2.dataInt,\
x.signals.exSignals.aluOut.dataInt


10
add
10


(2, 0, 10, 70, 80)

In [1361]:
x.exStage()
x.signals.exSignals.fwdA, x.signals.exSignals.fwdB ,\
x.signals.exSignals.aluIn1.dataInt, x.signals.exSignals.aluIn2.dataInt,\
x.signals.exSignals.aluOut.dataInt


A : Ex Hazard


(2, 0, 10, 70, 80)

In [1283]:
x.signals.exSignals.aluIn1.dataInt

80

In [1358]:
class idexR():
    def __init__(self, signals):
        self.pc = 0
        self.rd = 0
        
        self.control_wb_memtoReg = 0
        self.control_wb_regWrite = 0

        self.control_mem_memRead = 0 
        self.control_mem_memWrite = 0

        self.control_ex_aluOp = 0b00  # 0b00
        self.control_ex_aluSrc = 0

        self.readData1 = registerRISC(0)
        self.readData2 = registerRISC(0)
        self.func3 = 0b000
        self.func7bit30 = 0
        self.imm = registerRISC(0)
        
        
        self.rs1 = 0
        self.rs2 = 0
        self.instruction = registerRISC(0)

        #self.memOut = registerRISC(0)
        
    #def zeroEnforce(self):
    #    if self.rs1 == 0 : self.readData1 == registerRISC(0)
    #    if self.rs2 == 0 : self.readData2 == registerRISC(0)

        
        
class exSignals():
    def __init__(self):

        self.aluIn1 = registerRISC(0)
        self.aluIn2 = registerRISC(0)
        self.aluOut = registerRISC(0)
        self.aluZero = registerRISC(0)
        
        self.fwdA = 0b00
        self.fwdB = 0b00
        
        self.aluControlIn = 0b0000
        
        
        ####
        self.idexFlush = 0
        self.idexReset = 0        
        
    def __str__(self): #Use the print statement to show all attributes of the register
        return f"{vars(self)}"



        
    def __str__(self): #Use the print statement to show all attributes of the register
        return f"{vars(self)}"
    
'''
class exDev():
    def __init__(self, signals, registers):
        exAluControl.exec(self, signals, registers)
        exALU.exec(self, signals, registers)
        muxAB.exec(self, signals, registers)

'''

class exAluControl(): 
    @staticmethod
    def exec(self):
        
        #case LW or SW, ALU op = add
        if self.registers.idexR.control_ex_aluOp == 0b00 :
            self.signals.exSignals.aluControlIn = 0b0010
            
        #case beq , ALU op = subtract
        elif self.registers.idexR.control_ex_aluOp == 0b01:
            self.signals.exSignals.aluControlIn = 0b0110

        #case r-type add, ALU op = add
        elif self.registers.idexR.control_ex_aluOp == 0b10 and \
        self.registers.idexR.func3 == 0b000 and \
        self.registers.idexR.func7bit30 == 0  \
        :
            self.signals.exSignals.aluControlIn = 0b0010
            print("case2")

        #case r-type add, ALU op = sub
        elif self.registers.idexR.control_ex_aluOp == 0b10 and \
        self.registers.idexR.func3 == 0b000 and \
        self.registers.idexR.func7bit30 == 1  \
        :
            self.signals.exSignals.aluControlIn = 0b0110
        
        #case r-type and, ALU op = and
        elif self.registers.idexR.control_ex_aluOp == 0b10 and \
        self.registers.idexR.func3 == 0b111 and \
        self.registers.idexR.func7bit30 == 0  \
        :
            self.signals.exSignals.aluControlIn = 0b0000

        #case r-type or, ALU op = or
        elif self.registers.idexR.control_ex_aluOp == 0b10 and \
        self.registers.idexR.func3 == 0b111 and \
        self.registers.idexR.func7bit30 == 0  \
        :
            self.signals.exSignals.aluControlIn = 0b0001
            
        #emulate 0 -controls as if it is a beq, the op is add but nothing will be done in other stages
        # check here for ALU out if it will cause problems or not, or if we need to put an explicit nop
        #  for cases where ALUop is forbidden , like 11
        else :
            self.signals.exSignals.aluControlIn = 0b0000
    
    
class exForwarder(): 
    @staticmethod
    def exec(self):
        if self.registers.exmemR.control_wb_regWrite == 1 \
        and self.registers.exmemR.rd != 0 \
        and (self.registers.exmemR.rd == self.registers.idexR.rs1 ):
            self.signals.exSignals.fwdA = 0b10
            print ("A : Ex Hazard")
            
        elif self.registers.memwbR.control_wb_regWrite == 1 \
        and self.registers.memwbR.rd != 0 \
        and not (
            (self.registers.exmemR.control_wb_regWrite == 1) \
            and (self.registers.exmemR.rd != 0)
            and (self.registers.exmemR.rd == self.registers.idexR.rs1)
        ) and self.registers.memwbR.rd == self.registers.idexR.rs1         :
            self.signals.exSignals.fwdA = 0b01
            print ("A : MEM Hazard")
        else:
            self.signals.exSignals.fwdA = 0b00
        
        if self.registers.exmemR.control_wb_regWrite == 1 \
        and self.registers.exmemR.rd != 0 \
        and (self.registers.exmemR.rd == self.registers.idexR.rs2 ):
            self.signals.exSignals.fwdB = 0b10
            print ("B : Ex Hazard")
            
        elif self.registers.memwbR.control_wb_regWrite == 1 \
        and self.registers.memwbR.rd != 0 \
        and not (
            (self.registers.exmemR.control_wb_regWrite == 1) \
            and (self.registers.exmemR.rd != 0)
            and (self.registers.exmemR.rd == self.registers.idexR.rs2)
        ) and self.registers.memwbR.rd == self.registers.idexR.rs2         :
            self.signals.exSignals.fwdB = 0b01
            print ("B : MEM Hazard")
        else : 
            self.signals.exSignals.fwdB = 0b00
            
class muxABC(): 
    @staticmethod
    def exec(self):
        self.signals.exSignals.aluIn1 = {0b00 : self.registers.idexR.readData1
                                         ,0b10 : self.registers.exmemR.aluOut
                                         ,0b01 : self.registers.memwbR.memOut
                                        
                                        }.get(self.signals.exSignals.fwdA)
        
        self.signals.exSignals.aluIn2 = {0b00 : self.registers.idexR.readData2
                                         ,0b10 : self.registers.exmemR.aluOut
                                         ,0b01 : self.registers.memwbR.memOut
                                        
                                        }.get(self.signals.exSignals.fwdB)
        
        self.signals.exSignals.aluIn2 = {0b0 : self.signals.exSignals.aluIn2
                                         ,0b1 : self.registers.idexR.imm
                                        
                                        }.get(self.registers.idexR.control_ex_aluSrc)

        
class exALU():
    @staticmethod
    def exec(self):
        #print(self.signals.exSignals.aluIn1.dataInt)
        if self.signals.exSignals.aluControlIn == 0b0010:
            self.signals.exSignals.aluOut = self.signals.exSignals.aluIn1 + self.signals.exSignals.aluIn2
            #print("add")
            
        elif self.signals.exSignals.aluControlIn == 0b0110:
            self.signals.exSignals.aluOut = self.signals.exSignals.aluIn1 - self.signals.exSignals.aluIn2
            self.signals.exSignals.aluZero = 1 if self.signals.exSignals.aluOut.dataInt == 0 else 1
            #print("sub")
        
        elif self.signals.exSignals.aluControlIn == 0b0000:
            self.signals.exSignals.aluOut = self.signals.exSignals.aluIn1 & self.signals.exSignals.aluIn2
            #print("and")
        
        elif self.signals.exSignals.aluControlIn == 0b0001:
            self.signals.exSignals.aluOut = self.signals.exSignals.aluIn1 | self.signals.exSignals.aluIn2
            #print("or")
        
        else :
            self.signals.exSignals.aluOut = 0 ###Test this to indicate nop as well, should we put -ve ?
            #print("reset")
        
        #print(self.signals.exSignals.aluIn1.dataInt)
        #return self.signals.exSignals.aluOut


In [1359]:
class memSignals():
    def __init__(self):

        self.memReadData= registerRISC(0)
        self.pc = 0
        
        
    def __str__(self): #Use the print statement to show all attributes of the register
        return f"{vars(self)}"


class exmemR():
    def __init__(self, ifSignals):
        self.pc = 0
        self.rd = 0
        
        self.control_wb_memtoReg = 0
        self.control_wb_regWrite = 0

        self.control_mem_memRead = 0 
        self.control_mem_memWrite = 0

        self.r2Data = registerRISC(0)
        self.aluOut = registerRISC(0)
        self.aluZero = registerRISC(0)
        #self.memOut = registerRISC(0)

        
    def __str__(self): #Use the print statement to show all attributes of the register
        return f"{vars(self)}"


In [1356]:
class wbSignals():
    def __init__(self):
        #self.pc = 0

        self.writeData= registerRISC(0)
        #self.rd = 0
        #self.Rs= 0 
        #self.Rt= 0 
        
        
    def __str__(self): #Use the print statement to show all attributes of the register
        return f"{vars(self)}"


class memwbR():
    def __init__(self, ifSignals):
        self.pc = 0
        self.rd = 0
        self.control_wb_memtoReg = 0 #For test let it be 1,1,2,3 , 1,0,2,3 , 0,1,2,3 , 0,0,2,3
        self.control_wb_regWrite = 0
        self.memOut = registerRISC(0)
        self.aluOut = registerRISC(0)
        
    def __str__(self): #Use the print statement to show all attributes of the register
        return f"{vars(self)}"


In [1357]:
class ifSignals():
    def __init__(self):
        self.pc = 0
        self.pc4 = 0
        self.pcn1= 0
        self.pcImm = 0
        self.pcSrc = 0
        
        self.pcWrite = 0 ### Initial
        self.ifidWrite = 0 ### Initial
        
        self.ifidFlush = 0
        self.ifidControl = 0
        self.ifidComparatorOut = 0
        
    def __str__(self): #Use the print statement to show all attributes of the register
        return f"{vars(self)}"


class ifidR():
    def __init__(self, ifSignals):
        self.pc = 0
        self.instruction = registerRISC(data=0)
        
    def __str__(self): #Use the print statement to show all attributes of the register
        return f"{vars(self)}"

        
class ifDev():        
    def __init__(self,signals):
        self.ifMux1 = mux2in (signals.ifSignals.pc4
                                , signals.ifSignals.pcImm 
                                , signals.ifSignals.pcn1
                                , signals.ifSignals.pcSrc)
        
    def __str__(self): #Use the print statement to show all attributes of the register
        return f"{vars(self)}"

